In [1]:
import pandas as pd
from google.cloud import bigquery

In [2]:
# Initialize the BigQuery client using Application Default Credentials (ADC)
client = bigquery.Client()

# Test the connection by listing your BigQuery datasets
# datasets = list(client.list_datasets())
# project = client.project

# if datasets:
    # print(f"Datasets in project {project}:")
    # for dataset in datasets:
        # print(f"\t{dataset.dataset_id}")
# else:
    # print(f"No datasets found in project {project}.")

## Patient Query and Baseline Feature Queries

This query retrieves the basic information of all patients that were diagnosed with Sepsis. Sepsis is identified with ICD9 CODE `99591` and `99592`.

In [3]:
patientQuery = """
SELECT DISTINCT
    p.SUBJECT_ID,
    p.GENDER,
    p.DOB,
    p.DOD,
    a.HOSPITAL_EXPIRE_FLAG,
    a.HADM_ID,
    a.ADMITTIME,
    a.DISCHTIME,
    a.ADMISSION_TYPE,
    a.ETHNICITY,
    DATE_DIFF(DATE(a.ADMITTIME), DATE(p.DOB), YEAR) AS AGE_AT_ADMISSION
FROM
    mimiciii.patients p
JOIN
    mimiciii.diagnoses_icd d
    ON p.SUBJECT_ID = d.SUBJECT_ID
JOIN
    mimiciii.admissions a
    ON d.HADM_ID = a.HADM_ID
WHERE
    d.ICD9_CODE IN ('99591', '99592');
"""

patient_df = client.query(patientQuery).to_dataframe()
patient_df

,SUBJECT_ID,GENDER,DOB,DOD,HOSPITAL_EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ETHNICITY,AGE_AT_ADMISSION
0,18333,F,2094-01-21,2168-01-08,1,185644,2167-12-23 20:54:00,2168-01-08 02:40:00,EMERGENCY,WHITE,73
1,55935,F,2053-10-02,2112-02-08,1,116770,2112-01-30 23:04:00,2112-02-08 05:20:00,EMERGENCY,BLACK/AFRICAN AMERICAN,59
2,32012,F,2058-04-26,2144-02-08,1,138125,2144-02-01 05:13:00,2144-02-08 15:05:00,EMERGENCY,WHITE,86
3,8060,F,2111-11-26,2164-02-08,1,125303,2164-01-04 21:26:00,2164-02-08 12:00:00,EMERGENCY,WHITE,53
4,17634,F,2057-09-03,2116-02-16,1,169434,2116-02-14 17:30:00,2116-02-16 09:40:00,EMERGENCY,WHITE,59
...,...,...,...,...,...,...,...,...,...,...,...
5176,70239,M,2052-12-12,NaT,0,138364,2133-06-22 03:19:00,2133-06-23 18:00:00,EMERGENCY,WHITE,81
5177,70359,M,2087-11-22,NaT,0,122469,2168-11-18 16:10:00,2168-12-02 13:55:00,EMERGENCY,ASIAN - CHINESE,81
5178,43776,M,2069-06-06,NaT,0,177704,2144-08-31 05:53:00,2144-09-04 12:00:00,EMERGENCY,HISPANIC OR LATINO,75
5179,43925,M,2026-01-26,NaT,0,179764,2102-11-15 13:00:00,2102-11-29 18:04:00,ELECTIVE,WHITE,76


### Length of Stay (Days)

In [4]:
# 1. Ensure 'ADMITTIME' and 'DISCHTIME' are in datetime format
patient_df['ADMITTIME'] = pd.to_datetime(patient_df['ADMITTIME'])
patient_df['DISCHTIME'] = pd.to_datetime(patient_df['DISCHTIME'])

# 2. Calculate Length of Stay (LOS) in days
patient_df['LOS'] = (patient_df['DISCHTIME'] - patient_df['ADMITTIME']).dt.days

### Length of Stay in ICU (Days)

In [5]:
icu_los_query = """
SELECT 
    ICU.SUBJECT_ID, 
    AVG(ICU.LOS) AS LOS_ICU_MEAN
FROM 
    mimiciii.icustays AS ICU
INNER JOIN 
    mimiciii.diagnoses_icd AS D 
    ON D.SUBJECT_ID = ICU.SUBJECT_ID
WHERE 
    D.ICD9_CODE IN ('99591', '99592') 
    AND ICU.LOS IS NOT NULL
GROUP BY 
    ICU.SUBJECT_ID
ORDER BY 
    ICU.SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
icu_los_df = client.query(icu_los_query).to_dataframe()

# 5. Display the Cleaned ICU LOS DataFrame
icu_los_df.head()

dataframes_to_merge = [icu_los_df]

### Height Query

In [6]:
height_query = """
SELECT C.SUBJECT_ID, AVG(C.VALUENUM) AS HEIGHT_MEAN, ITEM.LABEL, ITEM.ITEMID
FROM mimiciii.chartevents AS C
INNER JOIN mimiciii.diagnoses_icd AS D ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN mimiciii.d_items AS ITEM ON ITEM.ITEMID = C.ITEMID
WHERE D.ICD9_CODE IN ('99591', '99592') AND (C.ITEMID  IN (226707, 226730, 1394))
GROUP BY C.SUBJECT_ID, ITEM.LABEL, ITEM.ITEMID
ORDER BY C.SUBJECT_ID ASC
"""

height_df = client.query(height_query).to_dataframe()

# Convert HEIGHT_MEAN to centimeters where ITEMID is not 226730
height_df['HEIGHT_CM'] = height_df.apply(
    lambda row: row['HEIGHT_MEAN'] * 2.54 if row['ITEMID'] != 226730 else row['HEIGHT_MEAN'],
    axis=1
)

# Remove duplicate records 
height_df_cleaned = height_df.drop_duplicates(subset=['SUBJECT_ID'])

# Select only relevant columns
height_df_cleaned = height_df_cleaned[['SUBJECT_ID', 'HEIGHT_CM']]
height_df_cleaned.head()

dataframes_to_merge.append(height_df_cleaned)

### Weight Query

In [7]:
# weight is in KG
weight_query = """
SELECT C.SUBJECT_ID, AVG(C.VALUENUM) AS WEIGHT_MEAN, ITEM.LABEL, ITEM.ITEMID
FROM mimiciii.chartevents AS C
INNER JOIN mimiciii.diagnoses_icd AS D ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN mimiciii.d_items AS ITEM ON ITEM.ITEMID = C.ITEMID
WHERE D.ICD9_CODE IN ('99591', '99592') AND (C.ITEMID  IN (226531, 763, 224639, 226512))
GROUP BY C.SUBJECT_ID, ITEM.LABEL, ITEM.ITEMID
ORDER BY C.SUBJECT_ID ASC
"""

weight_df = client.query(weight_query).to_dataframe()
weight_df.head()

max_weight = weight_df['WEIGHT_MEAN'].max()
weight_df_cleaned = weight_df[weight_df['WEIGHT_MEAN'] != max_weight]

# 1. Convert WEIGHT_MEAN from LB to KG where ITEMID is 226531
weight_df_cleaned.loc[weight_df_cleaned['ITEMID'] == 226531, 'WEIGHT_MEAN'] *= 0.453592

# 2. Remove duplicate records 
weight_df_cleaned = weight_df_cleaned.drop_duplicates(subset=['SUBJECT_ID'])
weight_df_cleaned = weight_df_cleaned.drop(['LABEL', 'ITEMID'], axis=1)
weight_df_cleaned.head()


dataframes_to_merge.append(weight_df_cleaned)

## Vital Signs Queries

### Heart Rate Query

In [8]:
heart_rate_query = """
SELECT C.SUBJECT_ID, AVG(C.VALUENUM) AS HEARTRATE_MEAN
FROM mimiciii.chartevents AS C
INNER JOIN mimiciii.diagnoses_icd AS D ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN mimiciii.d_items AS ITEM ON ITEM.ITEMID = C.ITEMID
WHERE D.ICD9_CODE IN ('99591', '99592') AND (C.ITEMID  IN (211, 220045))
GROUP BY C.SUBJECT_ID, ITEM.LABEL
ORDER BY C.SUBJECT_ID ASC
"""

heart_rate_df = client.query(heart_rate_query).to_dataframe()
heart_rate_df.head()
# merge = pd.merge(patient_df, heart_rate_df, how='left', on='SUBJECT_ID')
# merge.head()

dataframes_to_merge.append(heart_rate_df)


### Sys BP

In [9]:
sbp_query = """
SELECT 
    C.SUBJECT_ID, 
    AVG(C.VALUENUM) AS SBP_MEAN
FROM 
    mimiciii.chartevents AS C
INNER JOIN 
    mimiciii.diagnoses_icd AS D 
    ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN 
    mimiciii.d_items AS ITEM 
    ON ITEM.ITEMID = C.ITEMID
WHERE 
    D.ICD9_CODE IN ('99591', '99592') 
    AND C.ITEMID IN (51, 422, 455, 6701, 220050, 220179, 225309)
    AND C.VALUENUM IS NOT NULL
GROUP BY 
    C.SUBJECT_ID
ORDER BY 
    C.SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
sbp_df = client.query(sbp_query).to_dataframe()

# 5. Display the Aggregated SBP DataFrame
sbp_df.head()

dataframes_to_merge.append(sbp_df)

### Dialysis BP

In [10]:
dbp_query = """
SELECT 
    C.SUBJECT_ID, 
    AVG(C.VALUENUM) AS DBP_MEAN
FROM 
    mimiciii.chartevents AS C
INNER JOIN 
    mimiciii.diagnoses_icd AS D 
    ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN 
    mimiciii.d_items AS ITEM 
    ON ITEM.ITEMID = C.ITEMID
WHERE 
    D.ICD9_CODE IN ('99591', '99592') 
    AND C.ITEMID IN (8368, 8441, 8555, 220051, 220180, 225310)
    AND C.VALUENUM IS NOT NULL
GROUP BY 
    C.SUBJECT_ID
ORDER BY 
    C.SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
dbp_df = client.query(dbp_query).to_dataframe()

# 5. Display the Aggregated DBP DataFrame
dbp_df.head()

dataframes_to_merge.append(dbp_df)

### Mean Blood Pressure

In [11]:
map_query = """
SELECT 
    C.SUBJECT_ID, 
    AVG(C.VALUENUM) AS MAP_MEAN
FROM 
    mimiciii.chartevents AS C
INNER JOIN 
    mimiciii.diagnoses_icd AS D 
    ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN 
    mimiciii.d_items AS ITEM 
    ON ITEM.ITEMID = C.ITEMID
WHERE 
    D.ICD9_CODE IN ('99591', '99592') 
    AND C.ITEMID IN (52, 456, 6702, 220052, 220181, 225312)
    AND C.VALUENUM IS NOT NULL
GROUP BY 
    C.SUBJECT_ID
ORDER BY 
    C.SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
map_df = client.query(map_query).to_dataframe()

# 5. Display the Aggregated MAP DataFrame
map_df.head()

dataframes_to_merge.append(map_df)

### Resp Rate 

In [12]:
rr_query = """
SELECT 
    C.SUBJECT_ID, 
    AVG(C.VALUENUM) AS RR_MEAN
FROM 
    mimiciii.chartevents AS C
INNER JOIN 
    mimiciii.diagnoses_icd AS D 
    ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN 
    mimiciii.d_items AS ITEM 
    ON ITEM.ITEMID = C.ITEMID
WHERE 
    D.ICD9_CODE IN ('99591', '99592') 
    AND C.ITEMID IN (618, 224422, 224689, 224690, 220210)
    AND C.VALUENUM IS NOT NULL
GROUP BY 
    C.SUBJECT_ID
ORDER BY 
    C.SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
rr_df = client.query(rr_query).to_dataframe()

# 5. Display the Aggregated RR DataFrame
rr_df.head()

dataframes_to_merge.append(rr_df)

### Temperature

In [13]:
temp_query = """
WITH Temperature_Converted AS (
    SELECT
        C.SUBJECT_ID,
        C.CHARTTIME,
        -- Convert Fahrenheit to Celsius; leave Celsius measurements as is
        CASE 
            WHEN C.ITEMID IN (678, 679, 223761) THEN (C.VALUENUM - 32) * 5/9
            ELSE C.VALUENUM
        END AS TEMP_C
    FROM 
        mimiciii.chartevents AS C
    INNER JOIN 
        mimiciii.diagnoses_icd AS D 
        ON D.SUBJECT_ID = C.SUBJECT_ID
    WHERE 
        D.ICD9_CODE IN ('99591', '99592') 
        AND C.ITEMID IN (676, 677, 678, 679, 223762, 223761)
        AND C.VALUENUM IS NOT NULL
),

Temperature_Deduplicated AS (
    SELECT
        SUBJECT_ID,
        TIMESTAMP_TRUNC(CHARTTIME, HOUR) AS CHARTTIME_HOUR,
        AVG(TEMP_C) AS TEMP_C_Avg
    FROM 
        Temperature_Converted
    GROUP BY 
        SUBJECT_ID, CHARTTIME_HOUR
)

SELECT
    SUBJECT_ID,
    AVG(TEMP_C_Avg) AS TEMP_MEAN_C,
    MIN(TEMP_C_Avg) AS TEMP_MIN_C,
    MAX(TEMP_C_Avg) AS TEMP_MAX_C
FROM 
    Temperature_Deduplicated
GROUP BY 
    SUBJECT_ID
ORDER BY 
    SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
temp_df = client.query(temp_query).to_dataframe()

# 5. Display the Aggregated Temperature DataFrame
temp_df.head()

dataframes_to_merge.append(temp_df)

### Oxygen Saturation

In [14]:
oxygen_sat_query = """
WITH Oxygen_Saturation_Converted AS (
    SELECT
        C.SUBJECT_ID,
        C.CHARTTIME,
        C.ITEMID,
        C.VALUENUM AS OXYGEN_SAT
    FROM 
        mimiciii.chartevents AS C
    INNER JOIN 
        mimiciii.diagnoses_icd AS D 
        ON D.SUBJECT_ID = C.SUBJECT_ID
    WHERE 
        D.ICD9_CODE IN ('99591', '99592') 
        AND C.ITEMID IN (646, 834, 220227, 220277)
        AND C.VALUENUM IS NOT NULL
),

Oxygen_Saturation_Deduplicated AS (
    SELECT
        SUBJECT_ID,
        TIMESTAMP_TRUNC(CHARTTIME, MINUTE) AS CHARTTIME_MINUTE,
        AVG(OXYGEN_SAT) AS OXYGEN_SAT_Avg
    FROM 
        Oxygen_Saturation_Converted
    GROUP BY 
        SUBJECT_ID, CHARTTIME_MINUTE
)

SELECT
    SUBJECT_ID,
    AVG(OXYGEN_SAT_Avg) AS OXYGEN_SAT_MEAN,
    MIN(OXYGEN_SAT_Avg) AS OXYGEN_SAT_MIN,
    MAX(OXYGEN_SAT_Avg) AS OXYGEN_SAT_MAX
FROM 
    Oxygen_Saturation_Deduplicated
GROUP BY 
    SUBJECT_ID
ORDER BY 
    SUBJECT_ID ASC
"""

# 4. Execute the Query and Load Results into a DataFrame
oxygen_sat_df = client.query(oxygen_sat_query).to_dataframe()

# 5. Display the Aggregated Oxygen Saturation DataFrame

oxygen_sat_df.drop(['OXYGEN_SAT_MAX', 'OXYGEN_SAT_MIN'], axis=1, inplace=True)

oxygen_sat_df.head()

dataframes_to_merge.append(oxygen_sat_df)

## Accompanied Diseases

In [15]:
diabetes_query = """
SELECT DISTINCT
    SUBJECT_ID
FROM
    `physionet-data-435019.mimiciii.diagnoses_icd`
WHERE
    (
        icd9_code IN ("2535","3572","5881","7751","24900","24901","24910","24911","24920","24921","24930","24931","24940","24941","24950","24951","24960","24961","24970","24971","24980","24981","24990","24991","25000","25001","25002","25003","25010","25011","25012","25013","25020","25021","25022","25023","25030","25031","25032","25033","25040","25041","25042","25043","25050","25051","25052","25053","25060","25061","25062","25063","25070","25071","25072","25073","25080","25081","25082","25083","25090","25091","25092","25093","64800","64801","64802","64803","64804","V1221","V180","V771")
    )
    AND subject_id IN (
        SELECT SUBJECT_ID
        FROM `physionet-data-435019.mimiciii.diagnoses_icd`
        WHERE icd9_code IN ('99591', '99592') -- Sepsis codes
    )
ORDER BY SUBJECT_ID ASC
"""

# Execute the query and convert to DataFrame
df_diabetes_sepsis = client.query(diabetes_query).to_dataframe()

# Add 'DIABETES' column with value 1
df_diabetes_sepsis['DIABETES'] = 1

# Verify the DataFrame
df_diabetes_sepsis.head()


dataframes_to_merge.append(df_diabetes_sepsis)

## Common Sources of Infection

In [16]:
bio_query = """
SELECT * 
FROM `physionet-data-435019.mimiciii.microbiologyevents` m
JOIN `physionet-data-435019.mimiciii.diagnoses_icd` d
ON d.SUBJECT_ID = m.SUBJECT_ID
WHERE d.ICD9_CODE IN ('99591', '99592') 
ORDER BY d.SUBJECT_ID
"""

bio_df = client.query(bio_query).to_dataframe()
bio_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,...,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION,ROW_ID_1,SUBJECT_ID_1,HADM_ID_1,SEQ_NUM,ICD9_CODE
0,91,21,111970,2135-02-01,2135-02-01 12:06:00,70011,BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),<NA>,None,<NA>,...,None,None,None,<NA>,None,140,21,111970,11,99592
1,94,21,111970,2135-02-02,2135-02-02 12:15:00,70011,BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),<NA>,None,<NA>,...,None,None,None,<NA>,None,140,21,111970,11,99592
2,70,21,109451,2134-09-11,2134-09-11 09:35:00,70012,BLOOD CULTURE,<NA>,None,<NA>,...,None,None,None,<NA>,None,140,21,111970,11,99592
3,73,21,109451,2134-09-19,2134-09-19 15:55:00,70012,BLOOD CULTURE,<NA>,None,<NA>,...,None,None,None,<NA>,None,140,21,111970,11,99592
4,74,21,109451,2134-09-19,2134-09-19 16:20:00,70012,BLOOD CULTURE,<NA>,None,<NA>,...,None,None,None,<NA>,None,140,21,111970,11,99592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305623,631697,99991,151118,2184-12-27,2184-12-27 05:50:00,70079,URINE,<NA>,None,<NA>,...,None,None,None,<NA>,None,651001,99991,151118,5,99592
305624,631701,99991,151118,2184-12-28,2184-12-28 13:30:00,70079,URINE,<NA>,None,<NA>,...,None,None,None,<NA>,None,651001,99991,151118,5,99592
305625,631700,99991,151118,2184-12-28,2184-12-28 02:26:00,70091,MRSA SCREEN,80293,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,1,...,None,None,None,<NA>,None,651001,99991,151118,5,99592
305626,631703,99991,151118,2184-12-28,2184-12-28 21:45:00,70091,MRSA SCREEN,80293,POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS,1,...,None,None,None,<NA>,None,651001,99991,151118,5,99592


## Merging All Frames

In [17]:
for df in dataframes_to_merge:
    patient_df = pd.merge(patient_df, df, on='SUBJECT_ID', how='left')

In [18]:
# filling 0 for DIABETES encoded column
patient_df['DIABETES'] = patient_df['DIABETES'].fillna(0)

### Adding `LABEVENTS` Data 

In [19]:
patient_labevents = pd.read_csv('/Users/josh/Desktop/Macbook Working Files/Git Repos/650-DRAGON-SLAYERS/FILES/Josh-Dev/LABEVENTS-TABLE.csv')
 
patient_df = pd.merge(patient_df, patient_labevents, on='SUBJECT_ID', how='left')

## Cleaning

- Filter records by age (patient must be 18 or older)
  - Change patient age over 89 (obscured to value 300+) to median of 92 per DB documentation
- Drop columns with too many `null` values

In [20]:
# handle age cases: change 270 age to 92 (median). drop where age is less than 18

patient_df.loc[patient_df['AGE_AT_ADMISSION'] > 270, 'AGE_AT_ADMISSION'] = 92
patient_df = patient_df[patient_df['AGE_AT_ADMISSION'] >= 18].copy()

In [21]:
# dropping columns that have over 20% NULL values

patient_df.drop(columns=['HEIGHT_CM', 'WBC_MAX_VAL', 'WBC_MIN_VAL'], axis=1, inplace=True)

## Final Export

In [22]:
patient_df.to_csv('patient_df.csv')